In [1]:
from google.cloud import bigquery
import json
import os
import pandas as pd
import sys

sys.path.append(os.path.abspath("../govfund_grants/streamlit_dashboard/"))
from processing import tvl_col_to_df, chain_tvls_col_to_df, tokens_col_to_df, tokens_in_usd_col_to_df
from queries.defillama import query_tvl_data_from_bq

GCP_PROJECT_NAME = 'opensource-observer'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../oso_gcp_credentials.json'
client = bigquery.Client(GCP_PROJECT_NAME)

In [2]:
PROTOCOLS = ['velodrome', 'beefy', 'aave_v3']
CHAINS = [
    'Base', 'BOB', 'Fraxtal', 'Lisk', 'Mode', 'Optimism'
]
chain_pattern = '|'.join(f'^{chain}' for chain in CHAINS)

In [3]:
tvl_dfs = []
token_dfs = []
for protocol_slug in PROTOCOLS:
    chain_tvls_df, tvl_df, tokens_in_usd_df, tokens_df = query_tvl_data_from_bq(
        client=client,
        protocol=protocol_slug,
        bigquery_project_name=GCP_PROJECT_NAME
    )
    filtered_chain_tvls_df = (
        chain_tvls_df[chain_tvls_df['chain'].str.contains(chain_pattern, case=False, regex=True)]
        .assign(protocol=protocol_slug)
    )
    token_list = filtered_chain_tvls_df['token'].unique()
    filtered_tokens_in_usd_df = (
        tokens_in_usd_df[tokens_in_usd_df['token'].isin(token_list)]
        .assign(protocol=protocol_slug)
    )
    tvl_dfs.append(filtered_chain_tvls_df)
    token_dfs.append(filtered_tokens_in_usd_df)

In [5]:
df_tvl = (
    pd.concat(tvl_dfs, axis=0, ignore_index=True)
    .rename(columns={'value': 'token_value'})
)
df_tok = (
    pd.concat(token_dfs, axis=0, ignore_index=True)
    .rename(columns={'value': 'usd_value'})
    .drop('readable_date', axis=1)
)

df = (
    df_tvl.merge(df_tok, on=['date', 'protocol', 'token'])
    [['protocol', 'chain', 'token', 'readable_date', 'token_value', 'usd_value']]
)
df = df[df['readable_date'].between('2024-12-01', '2024-12-31')]
df

,protocol,chain,token,readable_date,token_value,usd_value
1229,velodrome,BOB,ETH,2024-12-01,3.190671e+01,1.188043e+05
1230,velodrome,BOB,RETH,2024-12-01,1.699360e+00,7.006609e+03
1231,velodrome,BOB,SOLVBTC,2024-12-01,0.000000e+00,0.000000e+00
1232,velodrome,BOB,SOLVBTC.BBN,2024-12-01,7.970000e-03,7.654630e+02
1233,velodrome,BOB,TBTC,2024-12-01,2.353240e+00,2.279145e+05
...,...,...,...,...,...,...
147509,aave_v3,Optimism-borrowed,USDC.E,2024-12-31,4.754860e+06,3.176073e+06
147510,aave_v3,Optimism-borrowed,USDT,2024-12-31,1.414365e+07,7.143464e+08
147511,aave_v3,Optimism-borrowed,WBTC,2024-12-31,3.704170e+01,3.588866e+09
147512,aave_v3,Optimism-borrowed,WETH,2024-12-31,1.899224e+04,1.123830e+09


In [6]:
token_mappings = (
    df
    .groupby('token', as_index=False)
    .agg({
        'chain': lambda x: ', '.join(sorted(x.unique())),
        'protocol': lambda x: ', '.join(sorted(x.unique())),
        'usd_value': lambda x: round(x.sum() / 30),
    })
    .sort_values(by='usd_value', ascending=False)
)
token_mappings.to_csv('data/defillama_token_listings.csv')
token_mappings

,token,chain,protocol,usd_value
94,WSTETH,"Base, Base-borrowed, Fraxtal, Optimism, Optimi...","aave_v3, velodrome",17674305493
85,WBTC,"BOB, Mode, Optimism, Optimism-borrowed","aave_v3, velodrome",7976433241
89,WETH,"Base, Base-borrowed, Lisk, Mode, Optimism, Opt...","aave_v3, velodrome",4282000497
81,USDT,"BOB, Base, Base-staking, Fraxtal, Lisk, Mode, ...","aave_v3, beefy, velodrome",2551659871
9,CBBTC,"Base, Base-borrowed",aave_v3,1524236617
...,...,...,...,...
50,PERP,Optimism,velodrome,0
19,ENS,Optimism,velodrome,0
23,FIS,Optimism,velodrome,0
33,KROM,Optimism,velodrome,0
